# Classification supervisé - labellisation à la main - SBERT
Schéma pour 4 casses et pour 10 classes
1. (Dédoublonner)
2. Nettoyer (déjà fait)
3. Vectoriser SBERT
5. train test split
6. Training et predict (4 classes)
7. Training et predict  (10 classes)
8. Évaluations

Ce même exercice a été  mené avec TD-IDF. Ce notebook présent uniquement la vectorisation SBERT

In [1]:
import pandas as pd

import numpy as np


In [2]:
dfbrut = pd.read_excel(r'D:\Documents\DATA_SCIENCE\DATASCIENTEST_juin_2024\PROJET\SAMPLE_838_codage_XL.xlsx', index_col=0)

#dfbrut = dfbrut.drop('index', axis = 1)


In [3]:
dfbrut=dfbrut.reset_index()

In [4]:
dfbrut.columns

Index(['index_838', 'index_1000', 'text', 'sign_count', 'Topic_gran_3',
       'Topic_gran_10', 'text_prepr', 'has_similarity', 'similar_with',
       'similarity_group', 'similarity_score'],
      dtype='object')

# 1. Vectorisation SBERT

In [5]:
# ------------------------------------------------------
# 3. CORPUS REUNI EN LISTE EXTERIEUR : chaque article devient un 'document' dans une liste all_docs
# --------------------------------------------------------
all_docs=  [doc for doc in dfbrut['text_prepr']]


In [6]:
from sentence_transformers import SentenceTransformer

c:\Users\wf\anaconda3\envs\AUG24\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [8]:
model = SentenceTransformer('all-MiniLM-L6-v2')

# Step 2: Encode the documents to obtain embeddings for each multi-sentence document
document_embeddings = model.encode(all_docs)

c:\Users\wf\anaconda3\envs\AUG24\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [12]:
print(document_embeddings.shape)
type(document_embeddings)

(838, 384)


numpy.ndarray

In [10]:
# transformer en liste de 838 vécteurs
List_doc_embeddings = document_embeddings.tolist()

In [11]:
len(List_doc_embeddings)

838

In [13]:
dfbrut.loc[:,'BERT_embedding']=List_doc_embeddings

In [14]:
dfbrut.columns

Index(['index_838', 'index_1000', 'text', 'sign_count', 'Topic_gran_3',
       'Topic_gran_10', 'text_prepr', 'has_similarity', 'similar_with',
       'similarity_group', 'similarity_score', 'BERT_embedding'],
      dtype='object')

# 2. Préparation

In [15]:
y = dfbrut['Topic_gran_3']
X = List_doc_embeddings

In [16]:
dfbrut.columns

Index(['index_838', 'index_1000', 'text', 'sign_count', 'Topic_gran_3',
       'Topic_gran_10', 'text_prepr', 'has_similarity', 'similar_with',
       'similarity_group', 'similarity_score', 'BERT_embedding'],
      dtype='object')

In [11]:
#dfbrut.to_excel('D:\Documents\DATA_SCIENCE\DATASCIENTEST_juin_2024\PROJET\SAMPLE_838_codage_XL.xlsx', index = False)

In [17]:
# supprimer des colonnes inutiles (issu du dédoublonnnemnt)

df = dfbrut.drop(['has_similarity', 'similar_with','similarity_group', 'similarity_score'], axis =1)



In [18]:
df.head(3)

,index_838,index_1000,text,sign_count,Topic_gran_3,Topic_gran_10,text_prepr,BERT_embedding
0,0,0,Social Media Claims Pastor Joel Osteen Won't O...,151,climate_events,climate_events,Social Media Claims Pastor Joel Osteen Won t O...,"[0.056134048849344254, 0.027818074449896812, 0..."
1,1,1,COVID-19 vaccine could be ready by December — ...,89,other,other,COVID 19 vaccine could be ready by December U...,"[-0.033955372869968414, -0.016457127407193184,..."
2,2,2,Suicide bomber kills 15 in Saudi security site...,86,International,International,Suicide bomber kills 15 in Saudi security site...,"[0.08185376971960068, 0.1165524572134018, -0.0..."


# 3. Apprentissage supervisé

## a. 4 Classes (3+1) : GridSearch

In [19]:
from sklearn.model_selection import train_test_split, KFold, cross_validate
X_train4, X_test4, y_train4, y_test4 = train_test_split(List_doc_embeddings, y, random_state = 123, test_size = 0.2)
y_train4.value_counts()

Topic_gran_3
climate_events    302
US_politics       156
other             107
International     105
Name: count, dtype: int64

In [20]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [21]:
clf1 = KNeighborsClassifier(n_neighbors=3)
clf2 = RandomForestClassifier(random_state=123)
clf3 = LogisticRegression(max_iter=1000)
clf4 = LinearSVC(max_iter=10000, penalty = 'l2')

In [22]:
param_grid_knn = {'n_neighbors': [5, 7, 9, 11]}
param_grid_rf = {'n_estimators': [20, 100, 200]}
param_grid_lr = {'C': [0.1, 1, 10, 100]}
param_grid_svc = {'C': [1, 5, 10, 15]}

In [24]:
grid_search_knn = GridSearchCV(clf1, param_grid_knn, cv=5)
grid_search_rf = GridSearchCV(clf2, param_grid_rf, cv=5)
grid_search_lr = GridSearchCV(clf3, param_grid_lr, cv=5)
grid_search_svc = GridSearchCV(clf4, param_grid_svc, cv=5)

In [25]:
grid_search_knn.fit(X_train4, y_train4)
grid_search_rf.fit(X_train4, y_train4)
grid_search_lr.fit(X_train4, y_train4)
grid_search_svc.fit(X_train4, y_train4)

GridSearchCV(cv=5, estimator=LinearSVC(max_iter=10000),
             param_grid={'C': [1, 5, 10, 15]})

In [27]:
# Meilleurs scores 4 classes
print("Meilleur score KNN:", grid_search_knn.best_score_)
print("Meilleur score Random Forest :", grid_search_rf.best_score_)
print("Meilleur score Régression Logistique:", grid_search_lr.best_score_)
print("Meilleur score LinearSVC Score:", grid_search_svc.best_score_)
print('----------------------')

# Meilleurs paramètres 4 classes
print("Meilleurs param. KNN :", grid_search_knn.best_params_)
print("Meilleurs param. Random Forest:", grid_search_rf.best_params_)
print("Meilleurs param. Régression Logistique:", grid_search_lr.best_params_)
print("Meilleurs param. LinearSVC :", grid_search_svc.best_params_)

Meilleur score KNN: 0.8194029850746268
Meilleur score Random Forest : 0.8074626865671644
Meilleur score Régression Logistique: 0.8597014925373134
Meilleur score LinearSVC Score: 0.8432835820895523
----------------------
Meilleurs param. KNN : {'n_neighbors': 5}
Meilleurs param. Random Forest: {'n_estimators': 200}
Meilleurs param. Régression Logistique: {'C': 1}
Meilleurs param. LinearSVC : {'C': 1}


In [28]:
# Meilleur modèle avec meilleur paramètre

lr_best = LogisticRegression(C= 1)


lr_best.fit(X_train4,y_train4)
y_pred_best_lr4=lr_best.predict(X_test4)
MXconf_lr = pd.crosstab(y_test4,y_pred_best_lr4, rownames = ['test set'], colnames = ['predicted'])
print('Accuracy LogReg : ',lr_best.score(X_test4, y_test4))
MXconf_lr


Accuracy LogReg :  0.8511904761904762


predicted,International,US_politics,climate_events,other
test set,,,,
International,20,4,0,0
US_politics,3,30,4,3
climate_events,0,0,84,1
other,1,5,4,9


## d. Gridsearch 10 classes TF IDF :

In [29]:
y10 = dfbrut['Topic_gran_10']
X = List_doc_embeddings

X_train10, X_test10, y_train10, y_test10 = train_test_split(List_doc_embeddings, y10, random_state = 123, test_size = 0.2)
y_train10.value_counts()

Topic_gran_10
climate_events                   302
inclassable                       67
US_politics_elections             66
US_politics                       62
international_politics_geopol     50
International                     30
US_politics_social_equality       28
other                             28
international_relations           25
safety_crime_immigraiton          12
Name: count, dtype: int64

In [30]:

from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [31]:
clf1 = KNeighborsClassifier(n_neighbors=3)
clf2 = RandomForestClassifier(random_state=123)
clf3 = LogisticRegression(max_iter=1000)
clf4 = LinearSVC(max_iter=10000, penalty = 'l2')

In [32]:
param_grid_knn = {'n_neighbors': [5, 7, 9, 11]}
param_grid_rf = {'n_estimators': [20, 100, 200]}
param_grid_lr = {'C': [0.1, 1, 10, 100]}
param_grid_svc = {'C': [1, 5, 10, 15]}

In [33]:
grid_search_knn = GridSearchCV(clf1, param_grid_knn, cv=5)
grid_search_rf = GridSearchCV(clf2, param_grid_rf, cv=5)
grid_search_lr = GridSearchCV(clf3, param_grid_lr, cv=5)
grid_search_svc = GridSearchCV(clf4, param_grid_svc, cv=5)

In [34]:
grid_search_knn.fit(X_train10, y_train10)
grid_search_rf.fit(X_train10, y_train10)
grid_search_lr.fit(X_train10, y_train10)
grid_search_svc.fit(X_train10, y_train10)

GridSearchCV(cv=5, estimator=LinearSVC(max_iter=10000),
             param_grid={'C': [1, 5, 10, 15]})

In [35]:
## Meilleurs scores 10 classes
print("Meilleur score KNN:", grid_search_knn.best_score_)
print("Meilleur score Random Forest :", grid_search_rf.best_score_)
print("Meilleur score Régression Logistique:", grid_search_lr.best_score_)
print("Meilleur score LinearSVC Score:", grid_search_svc.best_score_)
print('----------------------')

# Meilleurs paramètres 10 classes
print("Meilleurs param. KNN :", grid_search_knn.best_params_)
print("Meilleurs param. Random Forest:", grid_search_rf.best_params_)
print("Meilleurs param. Régression Logistique:", grid_search_lr.best_params_)
print("Meilleurs param. LinearSVC :", grid_search_svc.best_params_)

Meilleur score KNN: 0.6865671641791045
Meilleur score Random Forest : 0.6194029850746269
Meilleur score Régression Logistique: 0.7238805970149255
Meilleur score LinearSVC Score: 0.7268656716417911
----------------------
Meilleurs param. KNN : {'n_neighbors': 7}
Meilleurs param. Random Forest: {'n_estimators': 100}
Meilleurs param. Régression Logistique: {'C': 10}
Meilleurs param. LinearSVC : {'C': 1}


In [36]:
#Meilleur modèle 10 classes

svc_best = LinearSVC(max_iter=10000, penalty = 'l2', C=1)

svc_best.fit(X_train10,y_train10)
y_pred_best_svc=svc_best.predict(X_test10)
MXconf_lr = pd.crosstab(y_test10,y_pred_best_svc, rownames = ['test set'], colnames = ['predicted'])
print('Accuracy SVC linear : ',svc_best.score(X_test10, y_test10))
MXconf_lr


Accuracy SVC linear :  0.7142857142857143


predicted,International,US_politics,US_politics_elections,US_politics_social_equality,climate_events,inclassable,international_politics_geopol,international_relations,other,safety_crime_immigraiton
test set,,,,,,,,,,
International,2,1,0,0,0,0,5,0,0,1
US_politics,0,7,5,2,0,1,0,1,0,0
US_politics_elections,1,2,10,0,0,1,1,1,1,0
US_politics_social_equality,0,1,0,2,3,0,1,0,0,0
climate_events,0,0,0,0,84,1,0,0,0,0
inclassable,0,0,0,0,4,6,1,0,0,0
international_politics_geopol,1,0,0,0,0,0,4,0,0,0
international_relations,0,0,4,0,0,1,1,4,0,0
other,0,0,1,0,1,2,0,0,0,0


## évaluations 

In [37]:
from sklearn.metrics import davies_bouldin_score, calinski_harabasz_score

# 4 CLASSES

# -> lower is better
dbi4 = davies_bouldin_score(X_train4, y_train4)
print("Davies-Bouldin Index (DBI) 4 classes:", np.round(dbi4, 3))



# -> larger = better
ch_index4 = calinski_harabasz_score(X_train4, y_train4)
print("Calinski-Harabasz Index 4 classes:", np.round(ch_index4, 3))
print('-----------------------------------------')



# 10 CLASSES

# -> lower is better
dbi10 = davies_bouldin_score(X_train10, y_train10)
print("Davies-Bouldin Index (DBI) 10 classes:", np.round(dbi10, 3))



# -> larger = better
ch_index10 = calinski_harabasz_score(X_train10, y_train10)
print("Calinski-Harabasz Index 10 classes:", np.round(ch_index10, 3))



Davies-Bouldin Index (DBI) 4 classes: 5.016
Calinski-Harabasz Index 4 classes: 23.263
-----------------------------------------
Davies-Bouldin Index (DBI) 10 classes: 5.759
Calinski-Harabasz Index 10 classes: 9.796
